In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import pandas as pd
from pycaret.classification import ClassificationExperiment
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv

from itertools import combinations
import plotly.express as px

<built-in method __dir__ of sqlite3.Connection object at 0x7f9d0348f840>


In [2]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'BTCUSDT'
interval = '1h'
# lightgbm  xgboost lr et rf
estimator = 'knn'
_compare_models = False

start_train_date = '2010-01-01'  # train < and test >=
start_test_date = '2023-01-01'  # train < and test >=

stop_loss = 1.0
regression_times = 0  # 24 * 30 * 2  # horas
times_regression_PnL = 24
# numeric_features=['open', 'high', 'low', 'volume', 'close', 'rsi']
numeric_features = ['open', 'high', 'low', 'volume', 'close', ]
normalize = True
use_gpu = False
tune_model = False

### Metadata

<code>
Field Name - Description</br>
open_time - Kline Open time in unix time format</br>
open - Open Price</br>
high - High Price</br>
low	- Low Price</br>
close	- Close Price</br>
volume - Volume</br>
close_time - Kline Close time in unix time format</br>
quote_volume - Quote Asset Volume</br>
count	- Number of Trades</br>
taker_buy_volume - Taker buy base asset volume during this period</br>
taker_buy_quote_volume - Taker buy quote asset volume during this period</br>
ignore - Ignore</br>
</code>

In [3]:
cols = myenv.all_klines_cols.copy()
cols.remove('ignore')
data = utils.get_data(symbol=symbol, save_database=False, interval=interval, tail=-1, columns=cols, parse_dates=False)
data = utils.parse_type_fields(data, parse_dates=True)
data = utils.adjust_index(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 53841 entries, 2017-08-17 04:00:00 to 2023-10-13 19:00:00
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     53841 non-null  datetime64[ns]
 1   open                          53841 non-null  float32       
 2   high                          53841 non-null  float32       
 3   low                           53841 non-null  float32       
 4   close                         53841 non-null  float32       
 5   volume                        53841 non-null  float32       
 6   close_time                    53841 non-null  int64         
 7   quote_asset_volume            53841 non-null  float32       
 8   number_of_trades              53841 non-null  int16         
 9   taker_buy_base_asset_volume   53841 non-null  float32       
 10  taker_buy_quote_asset_volume  53841 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol
ix_open_time,,,,,,,,,,,,
2017-08-17 04:00:00,2017-08-17 04:00:00,4261.479980,4313.620117,4261.319824,4308.830078,47.181007,1502945999999,2.023661e+05,171,35.160503,1.509525e+05,BTCUSDT
2017-08-17 05:00:00,2017-08-17 05:00:00,4308.830078,4328.689941,4291.370117,4315.319824,23.234917,1502949599999,1.003048e+05,102,21.448071,9.260828e+04,BTCUSDT
2017-08-17 06:00:00,2017-08-17 06:00:00,4330.290039,4345.450195,4309.370117,4324.350098,7.229691,1502953199999,3.128231e+04,36,4.802861,2.079532e+04,BTCUSDT
2017-08-17 07:00:00,2017-08-17 07:00:00,4316.620117,4349.990234,4287.410156,4349.990234,4.443249,1502956799999,1.924106e+04,25,2.602292,1.129135e+04,BTCUSDT
2017-08-17 08:00:00,2017-08-17 08:00:00,4333.319824,4377.850098,4333.319824,4360.689941,0.972807,1502960399999,4.239503e+03,28,0.814655,3.552747e+03,BTCUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-13 15:00:00,2023-10-13 15:00:00,26757.080078,26784.269531,26685.960938,26706.830078,1052.143188,1697212799999,2.812184e+07,-14775,484.090393,1.293999e+07,BTCUSDT
2023-10-13 16:00:00,2023-10-13 16:00:00,26706.830078,26807.099609,26701.380859,26796.730469,998.312744,1697216399999,2.672120e+07,-21905,570.853027,1.527824e+07,BTCUSDT
2023-10-13 17:00:00,2023-10-13 17:00:00,26796.730469,26798.310547,26685.000000,26768.259766,965.460999,1697219999999,2.582702e+07,-26789,374.039032,1.000487e+07,BTCUSDT


In [4]:
data = calc_utils.calc_RSI(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 53841 entries, 2017-08-17 04:00:00 to 2023-10-13 19:00:00
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     53841 non-null  datetime64[ns]
 1   open                          53841 non-null  float32       
 2   high                          53841 non-null  float32       
 3   low                           53841 non-null  float32       
 4   close                         53841 non-null  float32       
 5   volume                        53841 non-null  float32       
 6   close_time                    53841 non-null  int64         
 7   quote_asset_volume            53841 non-null  float32       
 8   number_of_trades              53841 non-null  int16         
 9   taker_buy_base_asset_volume   53841 non-null  float32       
 10  taker_buy_quote_asset_volume  53841 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi
ix_open_time,,,,,,,,,,,,,
2017-08-17 04:00:00,2017-08-17 04:00:00,4261.479980,4313.620117,4261.319824,4308.830078,47.181007,1502945999999,2.023661e+05,171,35.160503,1.509525e+05,BTCUSDT,NaN
2017-08-17 05:00:00,2017-08-17 05:00:00,4308.830078,4328.689941,4291.370117,4315.319824,23.234917,1502949599999,1.003048e+05,102,21.448071,9.260828e+04,BTCUSDT,NaN
2017-08-17 06:00:00,2017-08-17 06:00:00,4330.290039,4345.450195,4309.370117,4324.350098,7.229691,1502953199999,3.128231e+04,36,4.802861,2.079532e+04,BTCUSDT,NaN
2017-08-17 07:00:00,2017-08-17 07:00:00,4316.620117,4349.990234,4287.410156,4349.990234,4.443249,1502956799999,1.924106e+04,25,2.602292,1.129135e+04,BTCUSDT,NaN
2017-08-17 08:00:00,2017-08-17 08:00:00,4333.319824,4377.850098,4333.319824,4360.689941,0.972807,1502960399999,4.239503e+03,28,0.814655,3.552747e+03,BTCUSDT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-13 15:00:00,2023-10-13 15:00:00,26757.080078,26784.269531,26685.960938,26706.830078,1052.143188,1697212799999,2.812184e+07,-14775,484.090393,1.293999e+07,BTCUSDT,42.864460
2023-10-13 16:00:00,2023-10-13 16:00:00,26706.830078,26807.099609,26701.380859,26796.730469,998.312744,1697216399999,2.672120e+07,-21905,570.853027,1.527824e+07,BTCUSDT,49.660172
2023-10-13 17:00:00,2023-10-13 17:00:00,26796.730469,26798.310547,26685.000000,26768.259766,965.460999,1697219999999,2.582702e+07,-26789,374.039032,1.000487e+07,BTCUSDT,47.724243


In [5]:
data = calc_utils.calc_ema_periods(data, periods_of_time=[times_regression_PnL, 200])
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 53841 entries, 2017-08-17 04:00:00 to 2023-10-13 19:00:00
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     53841 non-null  datetime64[ns]
 1   open                          53841 non-null  float32       
 2   high                          53841 non-null  float32       
 3   low                           53841 non-null  float32       
 4   close                         53841 non-null  float32       
 5   volume                        53841 non-null  float32       
 6   close_time                    53841 non-null  int64         
 7   quote_asset_volume            53841 non-null  float32       
 8   number_of_trades              53841 non-null  int16         
 9   taker_buy_base_asset_volume   53841 non-null  float32       
 10  taker_buy_quote_asset_volume  53841 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_24p,ema_24p_diff,ema_200p,ema_200p_diff
ix_open_time,,,,,,,,,,,,,,,,,
2017-08-17 04:00:00,2017-08-17 04:00:00,4261.479980,4313.620117,4261.319824,4308.830078,47.181007,1502945999999,2.023661e+05,171,35.160503,1.509525e+05,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2017-08-17 05:00:00,2017-08-17 05:00:00,4308.830078,4328.689941,4291.370117,4315.319824,23.234917,1502949599999,1.003048e+05,102,21.448071,9.260828e+04,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2017-08-17 06:00:00,2017-08-17 06:00:00,4330.290039,4345.450195,4309.370117,4324.350098,7.229691,1502953199999,3.128231e+04,36,4.802861,2.079532e+04,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2017-08-17 07:00:00,2017-08-17 07:00:00,4316.620117,4349.990234,4287.410156,4349.990234,4.443249,1502956799999,1.924106e+04,25,2.602292,1.129135e+04,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2017-08-17 08:00:00,2017-08-17 08:00:00,4333.319824,4377.850098,4333.319824,4360.689941,0.972807,1502960399999,4.239503e+03,28,0.814655,3.552747e+03,BTCUSDT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-13 15:00:00,2023-10-13 15:00:00,26757.080078,26784.269531,26685.960938,26706.830078,1052.143188,1697212799999,2.812184e+07,-14775,484.090393,1.293999e+07,BTCUSDT,42.864460,26797.999479,-0.340210,27238.511270,-1.951947
2023-10-13 16:00:00,2023-10-13 16:00:00,26706.830078,26807.099609,26701.380859,26796.730469,998.312744,1697216399999,2.672120e+07,-21905,570.853027,1.527824e+07,BTCUSDT,49.660172,26797.897958,-0.004357,27234.115441,-1.606019
2023-10-13 17:00:00,2023-10-13 17:00:00,26796.730469,26798.310547,26685.000000,26768.259766,965.460999,1697219999999,2.582702e+07,-26789,374.039032,1.000487e+07,BTCUSDT,47.724243,26795.526903,-0.101760,27229.480062,-1.693827


In [6]:
data = utils.regression_PnL(
    data=data,
    label=myenv.label,
    diff_percent=float(stop_loss),
    max_regression_profit_and_loss=int(times_regression_PnL),
    drop_na=True,
    drop_calc_cols=True,
    strategy=None)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 53618 entries, 2017-08-25 11:00:00 to 2023-10-12 19:00:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     53618 non-null  datetime64[ns]
 1   open                          53618 non-null  float32       
 2   high                          53618 non-null  float32       
 3   low                           53618 non-null  float32       
 4   close                         53618 non-null  float32       
 5   volume                        53618 non-null  float32       
 6   close_time                    53618 non-null  int64         
 7   quote_asset_volume            53618 non-null  float32       
 8   number_of_trades              53618 non-null  int16         
 9   taker_buy_base_asset_volume   53618 non-null  float32       
 10  taker_buy_quote_asset_volume  53618 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_24p,ema_24p_diff,ema_200p,ema_200p_diff,status
ix_open_time,,,,,,,,,,,,,,,,,,
2017-08-25 11:00:00,2017-08-25 11:00:00,4331.669922,4361.279785,4313.000000,4361.270020,25.795115,1503662399999,1.118111e+05,216,8.989599,3.906715e+04,BTCUSDT,62.460861,4280.805082,1.879668,4132.685234,5.531144,CAI_1.0
2017-08-25 12:00:00,2017-08-25 12:00:00,4361.270020,4361.270020,4261.129883,4309.689941,110.998611,1503665999999,4.792662e+05,1891,7.918988,3.425532e+04,BTCUSDT,54.577419,4283.115871,0.620438,4134.718968,4.231750,SOBE_1.0
2017-08-25 13:00:00,2017-08-25 13:00:00,4311.689941,4345.000000,4297.660156,4312.000000,20.801018,1503669599999,8.978516e+04,188,4.482264,1.937958e+04,BTCUSDT,54.852245,4285.426601,0.620088,4136.752745,4.236348,SOBE_1.0
2017-08-25 14:00:00,2017-08-25 14:00:00,4303.819824,4340.000000,4291.520020,4305.000000,10.610266,1503673199999,4.580427e+04,139,6.107378,2.639153e+04,BTCUSDT,53.790161,4286.992473,0.420050,4138.679954,4.018674,SOBE_1.0
2017-08-25 15:00:00,2017-08-25 15:00:00,4321.500000,4422.290039,4321.500000,4394.359863,21.796762,1503676799999,9.534934e+04,189,5.421801,2.366765e+04,BTCUSDT,63.504879,4295.581865,2.299525,4141.604271,6.102843,SOBE_1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-12 15:00:00,2023-10-12 15:00:00,26701.929688,26734.580078,26621.759766,26675.000000,1004.569580,1697126399999,2.680761e+07,-9532,476.734497,1.272224e+07,BTCUSDT,36.571541,26839.153347,-0.611619,27363.369891,-2.515662,ESTAVEL
2023-10-12 16:00:00,2023-10-12 16:00:00,26674.990234,26804.990234,26566.150391,26673.359375,2043.483398,1697129999999,5.454212e+07,16040,930.360352,2.482997e+07,BTCUSDT,36.483753,26825.889829,-0.568594,27356.504114,-2.497193,ESTAVEL
2023-10-12 17:00:00,2023-10-12 17:00:00,26673.359375,26692.650391,26555.000000,26641.900391,1387.198730,1697133599999,3.691580e+07,14442,683.252319,1.818268e+07,BTCUSDT,34.760696,26811.170674,-0.631342,27349.393630,-2.586870,SOBE_1.0


In [7]:
perc_data_label = data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(data)
perc_data_label

,open_time,perc
status,,
CAI_1.0,22909,0.427263
ESTAVEL,6686,0.124697
SOBE_1.0,24023,0.448040


In [8]:
train_data = data[(data['open_time'] >= start_train_date) & (data['open_time'] < start_test_date)]
train_data = train_data.sort_values(myenv.date_features)
train_data

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_24p,ema_24p_diff,ema_200p,ema_200p_diff,status
ix_open_time,,,,,,,,,,,,,,,,,,
2017-08-25 11:00:00,2017-08-25 11:00:00,4331.669922,4361.279785,4313.000000,4361.270020,25.795115,1503662399999,1.118111e+05,216,8.989599,3.906715e+04,BTCUSDT,62.460861,4280.805082,1.879668,4132.685234,5.531144,CAI_1.0
2017-08-25 12:00:00,2017-08-25 12:00:00,4361.270020,4361.270020,4261.129883,4309.689941,110.998611,1503665999999,4.792662e+05,1891,7.918988,3.425532e+04,BTCUSDT,54.577419,4283.115871,0.620438,4134.718968,4.231750,SOBE_1.0
2017-08-25 13:00:00,2017-08-25 13:00:00,4311.689941,4345.000000,4297.660156,4312.000000,20.801018,1503669599999,8.978516e+04,188,4.482264,1.937958e+04,BTCUSDT,54.852245,4285.426601,0.620088,4136.752745,4.236348,SOBE_1.0
2017-08-25 14:00:00,2017-08-25 14:00:00,4303.819824,4340.000000,4291.520020,4305.000000,10.610266,1503673199999,4.580427e+04,139,6.107378,2.639153e+04,BTCUSDT,53.790161,4286.992473,0.420050,4138.679954,4.018674,SOBE_1.0
2017-08-25 15:00:00,2017-08-25 15:00:00,4321.500000,4422.290039,4321.500000,4394.359863,21.796762,1503676799999,9.534934e+04,189,5.421801,2.366765e+04,BTCUSDT,63.504879,4295.581865,2.299525,4141.604271,6.102843,SOBE_1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,2022-12-31 19:00:00,16577.779297,16590.060547,16565.099609,16570.140625,4044.433594,1672516799999,6.704605e+07,-14485,2067.921631,3.428087e+07,BTCUSDT,50.101242,16570.439403,-0.001803,16683.831144,-0.681441,ESTAVEL
2022-12-31 20:00:00,2022-12-31 20:00:00,16570.140625,16574.970703,16564.089844,16568.599609,2622.143555,1672520399999,4.344849e+07,25169,1311.381104,2.172951e+07,BTCUSDT,49.712753,16570.292219,-0.010215,16682.684562,-0.683852,ESTAVEL
2022-12-31 21:00:00,2022-12-31 21:00:00,16568.189453,16571.640625,16544.119141,16548.279297,3618.773926,1672523999999,5.992803e+07,-26007,1746.413330,2.892190e+07,BTCUSDT,44.781689,16568.531185,-0.122231,16681.347196,-0.797705,ESTAVEL


In [9]:
test_data = data[data['open_time'] >= start_test_date]
test_data = test_data.sort_values(myenv.date_features)
test_data

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_24p,ema_24p_diff,ema_200p,ema_200p_diff,status
ix_open_time,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,2023-01-01 00:00:00,16541.769531,16545.699219,16508.390625,16529.669922,4364.835938,1672534799999,72146296.0,18782,2179.947754,36032352.0,BTCUSDT,42.406311,16560.267798,-0.184767,16676.903422,-0.882859,ESTAVEL
2023-01-01 01:00:00,2023-01-01 01:00:00,16529.589844,16556.800781,16525.779297,16551.470703,3590.066650,1672538399999,59376756.0,-4516,1730.249023,28617416.0,BTCUSDT,47.948830,16559.564030,-0.048874,16675.655335,-0.744706,ESTAVEL
2023-01-01 02:00:00,2023-01-01 02:00:00,16551.470703,16559.769531,16538.140625,16548.189453,3318.840332,1672541999999,54919452.0,-15674,1611.123047,26660868.0,BTCUSDT,47.212383,16558.654064,-0.063197,16674.387018,-0.756835,ESTAVEL
2023-01-01 03:00:00,2023-01-01 03:00:00,16548.189453,16548.189453,16518.210938,16533.039062,4242.080566,1672545599999,70122544.0,6652,2096.092773,34649044.0,BTCUSDT,43.862534,16556.604864,-0.142335,16672.980570,-0.839331,ESTAVEL
2023-01-01 04:00:00,2023-01-01 04:00:00,16533.039062,16535.970703,16511.919922,16521.849609,4285.009277,1672549199999,70802648.0,-1537,2188.401855,36159820.0,BTCUSDT,41.519444,16553.824444,-0.193157,16671.476780,-0.897504,ESTAVEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-12 15:00:00,2023-10-12 15:00:00,26701.929688,26734.580078,26621.759766,26675.000000,1004.569580,1697126399999,26807608.0,-9532,476.734497,12722238.0,BTCUSDT,36.571541,26839.153347,-0.611619,27363.369891,-2.515662,ESTAVEL
2023-10-12 16:00:00,2023-10-12 16:00:00,26674.990234,26804.990234,26566.150391,26673.359375,2043.483398,1697129999999,54542120.0,16040,930.360352,24829970.0,BTCUSDT,36.483753,26825.889829,-0.568594,27356.504114,-2.497193,ESTAVEL
2023-10-12 17:00:00,2023-10-12 17:00:00,26673.359375,26692.650391,26555.000000,26641.900391,1387.198730,1697133599999,36915800.0,14442,683.252319,18182684.0,BTCUSDT,34.760696,26811.170674,-0.631342,27349.393630,-2.586870,SOBE_1.0


In [11]:
# BTCUSDT 1h best params: close,volume,quote_asset_volume,number_of_trades,rsi
# numeric_features = 'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'.split(',')
# numeric_features = 'close,quote_asset_volume,ema_24p,ema_200p'.split(',')
print(f'Numeric Features: {numeric_features} - size: {len(numeric_features)}\n')

combination_numeric_features = []
for size in range(1, len(numeric_features) + 1): 
	comb = map(list, combinations(numeric_features, size))	
	for c in comb:
		res = ''
		for j in c:
			res += f'{j},'
		combination_numeric_features.append(res[0:len(res) - 1])

print(f'Combination Numeric Features: \n{combination_numeric_features}')

Numeric Features: ['close', 'quote_asset_volume', 'ema_24p', 'ema_200p'] - size: 4

Combination Numeric Features: 
['close', 'quote_asset_volume', 'ema_24p', 'ema_200p', 'close,quote_asset_volume', 'close,ema_24p', 'close,ema_200p', 'quote_asset_volume,ema_24p', 'quote_asset_volume,ema_200p', 'ema_24p,ema_200p', 'close,quote_asset_volume,ema_24p', 'close,quote_asset_volume,ema_200p', 'close,ema_24p,ema_200p', 'quote_asset_volume,ema_24p,ema_200p', 'close,quote_asset_volume,ema_24p,ema_200p']


In [12]:
df_resultado_simulacao = pd.DataFrame()
for aux_numeric_features in combination_numeric_features:
  experiement = ClassificationExperiment()
  features = []
  features += ['open_time', myenv.label]
  features += aux_numeric_features.split(',')
  # print(f'features: {features}')
  setup = experiement.setup(
      data=train_data[features].copy(),
      train_size=myenv.train_size,
      target=myenv.label,
      numeric_features=aux_numeric_features.split(','),
      date_features=['open_time'],
      create_date_columns=["hour", "day", "month"],
      fold_strategy='timeseries',
      fold=3,
      session_id=123,
      normalize=normalize,
      use_gpu=use_gpu,
      verbose=False,
      n_jobs=20,
      log_experiment=False)

  if _compare_models:
    best = setup.compare_models()
    estimator = setup.pull().index[0]
    print('Estimator: ' + estimator)
  else:
    best = setup.create_model(estimator)

  if tune_model:
    best = setup.tune_model(best)

  # predict on test set
  # holdout_pred = setup.predict_model(best)
  # print('Holdout Score:', holdout_pred['prediction_score'].mean())
  # print('Holdout Score Group:\n', holdout_pred[[myenv.label, 'prediction_score']].groupby(myenv.label).mean())

  predict = setup.predict_model(best, data=test_data.drop(columns=[myenv.label]))
  predict[myenv.label] = test_data[myenv.label]
  predict['_score'] = predict['prediction_label'] == predict[myenv.label]
  # print('Predict Score Mean:', predict['_score'].mean())
  # print('Predict Score Mean Group:\n', predict[[myenv.label, '_score']].groupby(myenv.label).mean())

  final_model = setup.finalize_model(best)

  ajusted_test_data = test_data.drop(myenv.label, axis=1)
  df_final_predict, res_score = utils.validate_score_test_data(
      setup,
      final_model,
      myenv.label,
      test_data,
      ajusted_test_data)

  #df_final_predict.info()
  # print('Final Score Mean:', res_score.mean().values[0])
  # print('Final Score Group:\n', res_score)

  start_test_date = df_final_predict['open_time'].min()
  end_test_date = df_final_predict['open_time'].max()

  # print('Simule Trading:')
  # print(f'Min Data: {start_test_date}')
  # print(f'Max Data: {end_test_date}')
  saldo_inicial = 100.0
  saldo_final = utils.simule_trading_crypto2(df_final_predict, start_test_date, end_test_date, saldo_inicial, stop_loss)
  print(f'>>>> Saldo Final: {saldo_final} - features: {features}\n\n')

  result_simulado = {'symbol': symbol, 'interval': interval, 'estimator': estimator, 'stop_loss': stop_loss, 'times_regression_profit_and_loss': times_regression_PnL, 'features': features, 'final_value': saldo_final}

  if res_score is not None:
    result_simulado['score'] = ''
    for i in range(0, len(res_score.index.values)):
      result_simulado['score'] += f'[{res_score.index.values[i]}={res_score["_score"].values[i]:.2f}]'
  
  df_resultado_simulacao = pd.concat([df_resultado_simulacao, pd.DataFrame([result_simulado])], ignore_index=True)
  df_resultado_simulacao.sort_values('final_value', inplace=True)

  simulation_results_filename = f'./resultado_simulacao_{symbol}_{interval}.csv'
  df_resultado_simulacao.to_csv(simulation_results_filename, sep=';', index=False)




,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4935,0.5939,0.4935,0.4864,0.4875,0.1154,0.1160
1,0.5284,0.6397,0.5284,0.5255,0.5254,0.1810,0.1817
2,0.5449,0.6542,0.5449,0.5425,0.5425,0.2073,0.2078
Mean,0.5223,0.6293,0.5223,0.5181,0.5184,0.1679,0.1685
Std,0.0214,0.0257,0.0214,0.0235,0.0230,0.0386,0.0386


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

>>>> Saldo Final: 212.58710611465116 - features: ['open_time', 'status', 'close']




,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4784,0.5675,0.4784,0.4700,0.4705,0.0846,0.0853
1,0.5030,0.6005,0.5030,0.4976,0.4972,0.1314,0.1324
2,0.5042,0.6045,0.5042,0.4987,0.4979,0.1281,0.1292
Mean,0.4952,0.5908,0.4952,0.4887,0.4885,0.1147,0.1157
Std,0.0119,0.0166,0.0119,0.0133,0.0127,0.0213,0.0215


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

>>>> Saldo Final: 584.0668449604868 - features: ['open_time', 'status', 'quote_asset_volume', 'ema_24p', 'ema_200p']




,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5004,0.6027,0.5004,0.4964,0.4960,0.1291,0.1298
1,0.5338,0.6469,0.5338,0.5327,0.5319,0.1922,0.1929
2,0.5412,0.6554,0.5412,0.5398,0.5397,0.2044,0.2048
Mean,0.5251,0.6350,0.5251,0.5230,0.5225,0.1752,0.1758
Std,0.0177,0.0231,0.0177,0.0190,0.0190,0.0330,0.0329


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

>>>> Saldo Final: 563.738422437264 - features: ['open_time', 'status', 'close', 'quote_asset_volume', 'ema_24p', 'ema_200p']


